# Functorial Narrative Analysis Tutorial

This notebook provides a hands-on introduction to the Functorial Narrative Analysis framework.

## What You'll Learn

1. How to apply observation functors to narrative text
2. How to detect Western (Harmon Circle) and Eastern (Kishōtenketsu) narrative structures
3. How to compare trajectories across different texts and cultures
4. How to visualize narrative shapes

## Setup

First, let's import the necessary modules and set up our environment.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Import functors
from src.functors import (
    SentimentFunctor,
    ArousalFunctor,
    EntropyFunctor,
    ThematicFunctor,
    EpistemicFunctor,
)

# Import detectors
from src.detectors import HarmonCircleDetector, KishotenketsuDetector

# Import visualization styling (JetBrains Mono)
from src.visualization import setup_style

# Apply project-wide styling with JetBrains Mono font
setup_style()

%matplotlib inline

## 1. Understanding Functors

A **functor** is a mapping that transforms one category into another while preserving structure.

In our framework:
- **Source Category**: Narrative states (text windows)
- **Target Category**: Numerical trajectories

Each functor measures a different dimension of the narrative:

| Functor | Measures | Range |
|---------|----------|-------|
| F_sentiment | Emotional valence | [-1, 1] |
| F_arousal | Tension/excitement | [0, 1] |
| F_entropy | Complexity/predictability | [0, 1] |
| F_thematic | Semantic drift | [0, 1] |
| F_epistemic | Certainty/uncertainty | [0, 1] |

## 2. Analyzing a Sample Text

Let's create a simple example text and analyze it.

In [ ]:
# Sample narrative text (beginning of a story)
sample_text = """
The morning sun cast long shadows across the peaceful village. Children played in the streets,
their laughter echoing off the stone walls. Old Mrs. Chen sat on her porch, watching the world
go by with a contented smile. Life was good, predictable, safe.

But something felt different today. A strange tension hung in the air, invisible but palpable.
The birds had stopped singing. Dogs whimpered and hid under porches. Even the children paused
their games, looking up at the sky with uncertain eyes.

Then the earth trembled. First a gentle vibration, then a violent shaking that sent tiles
crashing from rooftops. People screamed, running in all directions. The peaceful morning had
transformed into chaos in mere seconds. Terror gripped the village.

When the shaking finally stopped, an eerie silence descended. Dust clouds hung in the air.
Slowly, neighbors emerged from their hiding places. They looked at each other with shell-shocked
eyes, then began to help one another. Mrs. Chen, her house destroyed, found herself surrounded
by caring hands.

In the days that followed, something remarkable happened. The village that had always been
merely polite became genuinely connected. Strangers shared meals. Enemies reconciled. The
tragedy had revealed what truly mattered. The village rebuilt not just their homes, but their
community, stronger than before.
"""

print(f"Text length: {len(sample_text.split())} words")

### 2.1 Apply Sentiment Functor

In [ ]:
# Initialize sentiment functor
sentiment_functor = SentimentFunctor(method='vader')

# Process text with smaller windows for this short example
sentiment_trajectory = sentiment_functor.process_text(
    sample_text, 
    window_size=50,  # Smaller windows for short text
    overlap=25
)

print(f"Number of windows: {len(sentiment_trajectory.values)}")
print(f"Mean sentiment: {np.mean(sentiment_trajectory.values):.3f}")
print(f"Sentiment range: [{np.min(sentiment_trajectory.values):.3f}, {np.max(sentiment_trajectory.values):.3f}]")

In [ ]:
# Visualize sentiment trajectory
fig, ax = plt.subplots(figsize=(12, 4))

x = sentiment_trajectory.time_points
y = sentiment_trajectory.values

ax.plot(x, y, 'b-', linewidth=2, label='Sentiment')
ax.fill_between(x, y, alpha=0.3)
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

ax.set_xlabel('Narrative Time')
ax.set_ylabel('Sentiment (negative ← → positive)')
ax.set_title('Sentiment Trajectory')
ax.set_xlim(0, 1)
ax.set_ylim(-1, 1)
ax.legend()

plt.tight_layout()
plt.show()

### 2.2 Apply Multiple Functors

In [ ]:
# Initialize all functors
functors = {
    'Sentiment': SentimentFunctor(method='vader'),
    'Arousal': ArousalFunctor(),
    'Entropy': EntropyFunctor(),
}

# Extract all trajectories
trajectories = {}
for name, functor in functors.items():
    trajectories[name] = functor.process_text(sample_text, window_size=50, overlap=25)
    print(f"{name}: mean={np.mean(trajectories[name].values):.3f}")

In [ ]:
# Visualize all trajectories
fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

colors = ['blue', 'red', 'green']
for ax, (name, traj), color in zip(axes, trajectories.items(), colors):
    x = traj.time_points
    y = traj.values
    
    ax.plot(x, y, color=color, linewidth=2)
    ax.fill_between(x, y, alpha=0.2, color=color)
    ax.set_ylabel(name)
    ax.grid(True, alpha=0.3)

axes[-1].set_xlabel('Narrative Time')
fig.suptitle('Multi-Functor Trajectory Analysis', fontsize=14)
plt.tight_layout()
plt.show()

## 3. Detecting Narrative Structures

Our framework includes two structural detectors:

### Harmon Story Circle (Western)
8-stage circular journey: Comfort → Need → Go → Search → Find → Take → Return → Change

### Kishōtenketsu (Eastern)
4-act structure: Ki (起) → Shō (承) → Ten (転) → Ketsu (結)

In [ ]:
# Detect Harmon Circle
harmon_detector = HarmonCircleDetector()
harmon_match = harmon_detector.detect(
    sentiment_trajectory.values,
    trajectory_id="sample",
    title="Sample Story"
)

print("=== Harmon Circle Detection ===")
print(f"Conformance Score: {harmon_match.conformance_score:.2f}")
print(f"Pattern Type: {harmon_match.pattern_type}")
print(f"Nadir Position: {harmon_match.nadir_point:.1%}")
print(f"Notes: {', '.join(harmon_match.notes)}")

In [ ]:
# Detect Kishōtenketsu
kisho_detector = KishotenketsuDetector()
kisho_match = kisho_detector.detect(
    sentiment_trajectory.values,
    trajectory_id="sample",
    title="Sample Story"
)

print("=== Kishōtenketsu Detection ===")
print(f"Conformance Score: {kisho_match.conformance_score:.2f}")
print(f"Pattern Type: {kisho_match.pattern_type}")
print(f"Has Twist (Ten): {kisho_match.has_twist}")
print(f"Twist Position: {kisho_match.ten_position:.1%}")
print(f"Stability Score: {kisho_match.stability_score:.2f}")
print(f"Notes: {', '.join(kisho_match.notes)}")

## 4. Comparing Western vs Eastern Narrative Patterns

Let's create two contrasting texts to demonstrate the difference.

In [ ]:
# Western-style narrative (conflict-driven)
western_text = """
John had always dreamed of climbing Mount Everest. He was happy in his comfortable life,
but something was missing. An emptiness gnawed at him.

Finally, he quit his job and flew to Nepal. The journey was harder than he imagined.
The cold was brutal. His supplies ran low. Doubt crept in.

At base camp, disaster struck. An avalanche killed three climbers. John was terrified.
He wanted to give up, to go home, to forget this foolish dream.

But he pushed on. Through ice storms and near-death experiences. Through exhaustion
and despair. He lost feeling in his fingers but kept climbing.

Finally, he reached the summit. The world spread out below him like a promise.
He had done it. He had conquered his fear, conquered the mountain.

When John returned home, he was changed. The emptiness was gone, replaced by
a quiet confidence. He knew now what he was capable of achieving.
"""

# Eastern-style narrative (kishōtenketsu - juxtaposition, not conflict)
eastern_text = """
The old tea master lived alone in a small house by the river. Every morning
he prepared tea with precise, careful movements. His life was quiet and orderly.

He had few visitors, but those who came left feeling peaceful. The garden was
always perfect. The tatami mats were always clean. Time moved slowly here.

One day, a young businessman rushed in, late for an appointment elsewhere.
He demanded the fastest cup of tea possible. The master smiled and began
his usual slow preparation. The businessman grew impatient, checking his watch.

But something strange happened. As he watched the deliberate movements,
his urgency faded. The tea arrived. He tasted it slowly. For the first time
in years, he felt no need to hurry anywhere. He understood something new
about his own racing heart.
"""

print(f"Western text: {len(western_text.split())} words")
print(f"Eastern text: {len(eastern_text.split())} words")

In [ ]:
# Analyze both texts
western_sentiment = sentiment_functor.process_text(western_text, window_size=30, overlap=15)
eastern_sentiment = sentiment_functor.process_text(eastern_text, window_size=30, overlap=15)

# Detect structures
western_harmon = harmon_detector.detect(western_sentiment.values, "western", "Western Story")
eastern_harmon = harmon_detector.detect(eastern_sentiment.values, "eastern", "Eastern Story")

western_kisho = kisho_detector.detect(western_sentiment.values, "western", "Western Story")
eastern_kisho = kisho_detector.detect(eastern_sentiment.values, "eastern", "Eastern Story")

print("=== Structure Detection Comparison ===")
print(f"\nWestern Text:")
print(f"  Harmon Circle: {western_harmon.conformance_score:.2f} ({western_harmon.pattern_type})")
print(f"  Kishōtenketsu: {western_kisho.conformance_score:.2f} ({western_kisho.pattern_type})")

print(f"\nEastern Text:")
print(f"  Harmon Circle: {eastern_harmon.conformance_score:.2f} ({eastern_harmon.pattern_type})")
print(f"  Kishōtenketsu: {eastern_kisho.conformance_score:.2f} ({eastern_kisho.pattern_type})")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Western
ax = axes[0]
x = np.linspace(0, 1, len(western_sentiment.values))
ax.plot(x, western_sentiment.values, 'b-', linewidth=2)
ax.fill_between(x, western_sentiment.values, alpha=0.3)
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.set_title(f'Western Narrative\nHarmon: {western_harmon.conformance_score:.2f} | Kishōtenketsu: {western_kisho.conformance_score:.2f}')
ax.set_xlabel('Narrative Time')
ax.set_ylabel('Sentiment')
ax.set_xlim(0, 1)

# Eastern
ax = axes[1]
x = np.linspace(0, 1, len(eastern_sentiment.values))
ax.plot(x, eastern_sentiment.values, 'r-', linewidth=2)
ax.fill_between(x, eastern_sentiment.values, alpha=0.3, color='red')
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.set_title(f'Eastern Narrative\nHarmon: {eastern_harmon.conformance_score:.2f} | Kishōtenketsu: {eastern_kisho.conformance_score:.2f}')
ax.set_xlabel('Narrative Time')
ax.set_ylabel('Sentiment')
ax.set_xlim(0, 1)

plt.tight_layout()
plt.show()

## 5. Working with Real Texts

To analyze real literary texts, load them from the corpus.

In [ ]:
# Example: Loading a processed trajectory
import json

# This would load a real trajectory from the results
# trajectory_file = Path('../data/results/trajectories/sentiment/en/example_sentiment.json')

# if trajectory_file.exists():
#     with open(trajectory_file) as f:
#         data = json.load(f)
#     values = np.array(data['values'])
#     print(f"Loaded trajectory with {len(values)} points")

print("To load real trajectories, run the corpus collection and processing pipeline first:")
print("  make corpus-gutenberg")
print("  make preprocess")
print("  make extract-trajectories")

## 6. Key Takeaways

1. **Functors are composable**: You can apply multiple functors to get a multi-dimensional view of narrative structure.

2. **Structure is measurable**: The Harmon Circle and Kishōtenketsu detectors provide quantitative conformance scores.

3. **Cultural patterns differ**: Western narratives tend toward descent-ascent (valley) shapes, while Eastern narratives often show plateau-spike-resolution patterns.

4. **The framework is extensible**: New functors can be added by subclassing `BaseFunctor`.

## Next Steps

- Run `make replication` to reproduce the full research results
- Explore the `data/results/` directory for pre-computed analyses
- Read `REPLICATION.md` for detailed reproduction instructions
- Check open issues at https://github.com/ibrahimcesar/functorial-narrative-analysis/issues